# Elmo

In [1]:
#Load packages
import tensorflow as tf 
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences 
from sklearn import metrics, preprocessing,model_selection 
from sklearn.metrics import accuracy_score 
import keras
from keras.layers import Input, Lambda, Dense 
from keras.models import Model 
import keras.backend as K 
import matplotlib.pyplot as pit 
import seaborn as sns
import numpy as np 
import string 
import pandas as pd 
import re 
import spacy 
from nltk.corpus import stopwords 
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS 
from spacy.lang.en import English 
spacy.load('en') 
parser = English()

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_extraction.stop_words module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_extraction.text. Anything that cannot be imported from sklearn.feature_extraction.text is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##load Elmo

In [ ]:
# get elmo from tensorflow hub 
import tensorflow_hub as hub 
import tensorflow as tf 

embed = hub.load("https://tfhub.dev/google/elmo/2") 

# ELMo Embedding
def ELMoEmbedding(x): 
  return embed(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)["default"] 

## Pre-process Data

In [ ]:
# Stop words and special characters 
import nltk 
nltk.download('stopwords') 

STOPLIST = set(stopwords.words('english') + list(ENGLISH_STOP_WORDS)) 
SYMBOLS = " ".join(string.punctuation).split(" ") + ["-","...","”","''"]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
#Data Cleaner and tokenizer 
def tokenizeText(text): 
  text = text.strip().replace("\n", " ").replace("\r", " ") 
  text = text.lower() 
  
  tokens = parser(text) 
  #lematization
  lemmas = [] 
  for tok in tokens:
     lemmas.append(tok.lemma_.lower().strip() if tok.lemma_ != "-PRON-" else tok.lower_)
  tokens = lemmas 
  #reomve stop words and special charaters 
  tokens = [tok for tok in tokens if tok.lower() not in STOPLIST] 
  tokens = [tok for tok in tokens if tok not in SYMBOLS] 
  
  tokens = [tok for tok in tokens if len(tok) >= 3]
  
  #remove remaining tokens that are not alphabetic 
  tokens = [tok for tok in tokens if tok.isalpha()] 
  
  tokens = list(set(tokens)) 
  
  return ' '.join(tokens[:]) 

In [ ]:
def encode(le_enc, labels): 
  enc = le_enc.transform(labels) 
  return keras.utils.to_categorical(enc) 

def decode(le_enc, one_hot):
   dec = np.argmax(one_hot, axis =1) 
   return le_enc.inverse_transform(dec) 

In [ ]:
# load the dataset 
import pandas as pd
trainDF_Sheet_1 = pd.read_csv("/content/drive/My Drive/data/data_train.csv")

In [ ]:
trainDF_Sheet_1.head() 

,review,is_bad_review
0,I am so angry that i made this post available...,1
1,No Negative No real complaints the hotel was g...,0
2,Rooms are nice but for elderly a bit difficul...,0
3,My room was dirty and I was afraid to walk ba...,1
4,You When I booked with your company on line y...,0


In [ ]:
trainDF_Sheet_1.shape 

(515738, 2)

In [ ]:
trainDF_Sheet_1['review'].unique() 

array([' I am so angry that i made this post available via all possible sites i use when planing my trips so no one will make the mistake of booking this place I made my booking via booking com We stayed for 6 nights in this hotel from 11 to 17 July Upon arrival we were placed in a small room on the 2nd floor of the hotel It turned out that this was not the room we booked I had specially reserved the 2 level duplex room so that we would have a big windows and high ceilings The room itself was ok if you don t mind the broken window that can not be closed hello rain and a mini fridge that contained some sort of a bio weapon at least i guessed so by the smell of it I intimately asked to change the room and after explaining 2 times that i booked a duplex btw it costs the same as a simple double but got way more volume due to the high ceiling was offered a room but only the next day SO i had to check out the next day before 11 o clock in order to get the room i waned to Not the best way to 

In [ ]:
trainDF_Sheet_1['is_bad_review'].value_counts() 

0    493457
1     22281
Name: is_bad_review, dtype: int64

In [ ]:
# model parameters

vocab_size = 1000
embedding_dim = 256
max_length = 512
trunc_type='post'
padding_type='post'
oov_tok = "<00V>"
training_portion = .7

In [ ]:
# train test split
# ---------- 
              
# proportion of training dataset
train_size = int(len(sentences) * training_portion)

# training dataset
train_sentences = sentences[:train_size]
train_labels = encoded_labels[:train_size]

# validation dataset
validation_sentences = sentences[train_size: ]
validation_labels = encoded_labels[train_size: ]

In [ ]:
# tokenizing, sequencing, padding features

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index
y_test = trainDF_Sheet_1["sentiment"]
train_sentences = trainDF_Sheet_1["review"]
train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_sequences, padding=padding_type, maxlen=max_length)
prediction = model.predict(train_padded)
y_pred = (prediction > 0.5)
validation_sequences = tokenizer.texts_to_sequences(validation_sentences)
validation_padded = pad_sequences(validation_sequences, padding=padding_type, maxlen=max_length)

NameError: ignored

## With cnn

In [ ]:
# model initialization
model = tf.keras.Sequential([
  tf.keras.layers.ELMoEmbedding(vocab_size, embedding_dim, input_length=max_length),
  tf.keras.layers.Conv1D(128, 5, activation='relu'),
  tf.keras.layers.GlobalMaxPooling1D(),
  tf.keras.layers.Dense(24, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid')
))

# compile model
model.compile(loss='binary_crossentropy’,
   optimizer='adam',
   metrics=['accuracy'])

# model summary
model.summary ()

SyntaxError: ignored

In [ ]:
# model fit
with tf.Session() as session:
  K.set_session(session)
  session.run(tf.global_variables_initializer())
  session.run(tf.tables_initializer())

num_epochs = 20

history = model. fit(train_padded, train_labels,
   epochs=num_epochs, verbose=1,
   validation_data=(validation_padded, validation_labels))

model.save_weights('./response-elmo-model.h5’)

In [ ]:
from sklearn.metrics import fi_score, precision_score, recall_score, confusion_matrix
print("F1-score: {0}".format(f1_score(y_pred, y_test)))
print("Precision: ")
precision_score(y_pred, y_test)
print("Recall: ")
recall_score(y_pred, y_test)
print("Confusion matrix: ")
confusion_matrix(y_pred, y_test)

In [ ]:
 # accuracy and loss

plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel("Epochs")
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel("Epochs")
plt.ylabel('Loss')
plt.legend()

plt.show()

## With RNN

In [ ]:
# model initialization
model = tf.keras.Sequential([
  tf.keras.layers.ELMoEmbedding(vocab_size, embedding_dim, input_length=max_length),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(100)),
  tf.keras.layers.Dense(24, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid' )
])

# compile model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model summary
model.summary()

In [ ]:
# model fit
with tf.Session() as session:
   K.set_session(session)
   session.run(tf.global_variables_initializer())
   session.run(tf.tables_initializer())
num_epochs = 20

history = model. fit(train_padded, train_labels,
                    epochs=num_epochs, verbose=1,
                    validation_data=(validation_padded, validation_labels))
model.save_weights('./response-elmo-model.h5')

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
print("F1-score: {0}".format(f1_score(y_pred, y_test)))
print("Precision: ")
precision_score(y_pred, y_test)
print("Recall: ")
recall_score(y_pred, y_test)
print("Confusion matrix: ")
confusion_matrix(y_pred, y_test)

In [ ]:
# accuracy and loss
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel("Epochs")
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel("Epochs")
plt.ylabel('Loss')
plt.legend()

plt.show()

## With RMDL

In [ ]:
# model initialization

model = tf.keras.Sequential([
  tf.keras.layers.ELMoEmbedding(vocab_size, embedding_dim, input_length=max_length),
  tf.keras.layers.Conv1D(128, 5, activation='relu'),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
  tf.keras.layers.Dense(24, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid' )
])
# compile model
model.compile(loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

# model summary
model. summary ()

In [ ]:
# model fit
with tf.Session() as session:
   K.set_session(session)
   session.run(tf.global_variables_initializer())
   session.run(tf.tables_initializer())
num_epochs = 20

history = model. fit(train_padded, train_labels,
                    epochs=num_epochs, verbose=1,
                    validation_data=(validation_padded, validation_labels))
model.save_weights('./response-elmo-model.h5')

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
print("F1-score: {0}".format(f1_score(y_pred, y_test)))
print("Precision: ")
precision_score(y_pred, y_test)
print("Recall: ")
recall_score(y_pred, y_test)
print("Confusion matrix: ")
confusion_matrix(y_pred, y_test)

In [ ]:
# accuracy and loss
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel("Epochs")
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel("Epochs")
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
# model initialization
model = tf.keras.Sequential([
    tf.keras.layers.ELMoEmbedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras._layers.LSTM(32)),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
# compile model
model.compile(loss='binary_crossentropy',
         optimizer='adam',
         metrics=['accuracy'])

# model summary
model.summary()

In [ ]:
# model fit
with tf.Session() as session:
   K.set_session(session)
   session.run(tf.global_variables_initializer())
   session.run(tf.tables_initializer())
num_epochs = 20

history = model. fit(train_padded, train_labels,
                    epochs=num_epochs, verbose=1,
                    validation_data=(validation_padded, validation_labels))
model.save_weights('./response-elmo-model.h5')

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
print("F1-score: {0}".format(f1_score(y_pred, y_test)))
print("Precision: ")
precision_score(y_pred, y_test)
print("Recall: ")
recall_score(y_pred, y_test)
print("Confusion matrix: ")
confusion_matrix(y_pred, y_test)

In [ ]:
# accuracy and loss
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel("Epochs")
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel("Epochs")
plt.ylabel('Loss')
plt.legend()

plt.show()